# Capitulo 8 - Referências a objetos, mutabilidade e reciclagem

## Variáveis não são caixas

In [1]:
# Variáveis a e b armazenam referências à mesma lista em vez de cópias
a = [1, 2, 3]
b = a
a.append(4)
b

[1, 2, 3, 4]

In [7]:
# Variáveis são atribuídas a objetos somente depois que os objetos são criados

class Gizmo:
  def __init__(self):
    print('Gizmo id: %d' % id(self))

x = Gizmo()
y = Gizmo() * 10
# dir()

Gizmo id: 139748774050352
Gizmo id: 139748774049584


TypeError: unsupported operand type(s) for *: 'Gizmo' and 'int'

## Identidade, igualdade e apelidos

In [14]:
# charles e lewis se referem ao mesmo objeto

charles = {'name': 'Charles L. Dodgson', 'born': 1832}
lewis = charles

lewis is charles
id(charles), id(lewis)
lewis['balance'] = 950
charles

{'name': 'Charles L. Dodgson', 'born': 1832, 'balance': 950}

In [18]:
# alex e charles são comparados como iguais, mas alex nào é charles
alex = {'name': 'Charles L. Dodgson', 'born': 1832, 'balance': 950}

alex == charles
alex is not charles

True

## Escolhendo entre `==` e `is`

O operador `=` compara valores de objetos (os dados que eles armazenam), enquanto `is` compara suas identidades.

## A relativa imutabilidade das tuplas

In [24]:
## t1 e t2 inicialmente são comparados como iguais, mas alterar um item mutável da tupla t1 a torna diferente

t1 = (1, 2, [30, 40])
t2 = (1, 2, [30, 40])

t1 == t2
id(t1[-1])
t1[-1].append(99)
t1
id(t1[-1])
t1 == t2

False

## Cópias são rasas por padrão

In [29]:
# Fazendo uma cópia rasa de uma lista que contém outra lista; copie e cole este código para ver sua animação em Online Python Tutor

l1 = [3, [66, 55, 44], (7, 8, 9)]
l2 = list(l1)

l1.append(100)
l1[1].remove(55)
print('l1:', l1)
print('l2:', l2)

l2[1] += [33, 22]
l2[2] += (10, 11)
print('l1:', l1)
print('l2:', l2)

l1: [3, [66, 44], (7, 8, 9), 100]
l2: [3, [66, 44], (7, 8, 9)]
l1: [3, [66, 44, 33, 22], (7, 8, 9), 100]
l2: [3, [66, 44, 33, 22], (7, 8, 9, 10, 11)]


## Cópias profundas e rasas de objetos quaisquer

In [32]:
# Bus pega e deixa passageiros

class Bus:
  def __init__ (self, passengers=None):
    if passengers is None:
      self.passengers = []
    else:
      self.passengers = list(passengers)
  
  def pick(self, name):
    self.passengers.append(name)
  
  def drop(self, name):
    self.passengers.remove(name)

In [42]:
# Efeitos do uso de copy versus deepcopy

import copy

bus1 = Bus(['Alice', 'Bill', 'Claire', 'David'])
bus2 = copy.copy(bus1)
bus3 = copy.deepcopy(bus1)

id(bus1), id(bus2), id(bus3)
bus1.drop('Bill')
bus2.passengers
id(bus1.passengers), id(bus2.passengers), id(bus3.passengers)
bus3.passengers

['Alice', 'Bill', 'Claire', 'David']

In [44]:
# Referências cíclicas: b referencia a e, em seguida, é concatenado a a; deepcopy ainda é capaz de copiar a

from copy import deepcopy

a = [10, 20]
b = [a, 30]
a.append(b)
a
c = deepcopy (a)
c

[10, 20, [[...], 30]]

## Parâmetros de função como referências

In [57]:
# Uma função pode alterar qualquer objeto mutável que ela receber

def f(a, b):
  a += b
  return a

x = 1
y = 2
f(x, y)
x, y

a = [1, 2]
b = [3, 4]
f(a, b)
a, b

t = (10, 20)
u = (30, 40)
f(t, u)
t, u

((10, 20), (30, 40))

## Tipos mutáveis como default de parâmetros: péssima ideia

In [59]:
# Uma classe simples para ilustrar o perigo de um default mutável

class HauntedBus:
  """Um modelo de ônibus assombrado por passageiros fantasmas"""
  def __init__(self, passengers=[]):
    self.passengers = passengers
  
  def pick(self, name):
    self.passengers.append(name)
  
  def drop(self, name):
    self.passengers.remove(name)

In [60]:
# ônibus assombrados por passageiros fantasmas
bus1 = HauntedBus(['Alice', 'Bill'])
bus1.passengers
bus1.pick('Charlie')
bus1.drop('Alice')
bus1.passengers

bus2 = HauntedBus()
bus2.pick('Carrie')
bus2.passengers

bus3 = HauntedBus()
bus3.passengers
bus3.pick('Dave')

bus2.passengers
bus2.passengers is bus3.passengers
bus1.passengers

['Bill', 'Charlie']

## Programação defensiva com parâmetros mutáveis

In [62]:
# Uma classe simples que mostra os perigos de alterar argumentos recebidos

class TwilightBus:
  """Um modelo de ônibus que faz os passageiros desaparecerem"""
  def __init__ (self, passengers=None):
    if passengers is None:
      self.passengers = []
    else:
      self.passengers = list(passengers)
  
  def pick(self, name):
    self.passengers.append(name)
  
  def drop(self, name):
    self.passengers.remove(name)

## `del` e coleta de lixo

In [71]:
# Observando o fim de um objeto quando não há mais referências a ele
import weakref

s1 = {1, 2, 3}
s2 = s1

def bye():
  print('Gone with the wind...')

ender = weakref.finalize(s1, bye)
ender.alive
del s1
ender.alive
s2 = 'spam'
ender.alive

Gone with the wind...


False

## Referências fracas

In [23]:
# Uma referência fraca é um invocável que devolve o objeto referenciado ou None se o referente não existir
import weakref

a_set = {0, 1}
wref = weakref.ref(a_set)
wref
wref()
a_set = {2,3,4}
wref()
wref() is None
wref() is None

True

## Esquete com WeakValueDictionary

In [24]:
# Cheese tem um atributo kind e uma representação-padrão

class Cheese:  
  def __init__(self, kind):
    self.kind = kind
  
  def __repr__(self):
    return 'Cheese(%r)' % self.kind

In [30]:
# Cliente: "Afinal de contas, você tem algum tipo de queijo aqui?"

import weakref

stock = weakref.WeakValueDictionary()
catalog = [Cheese('Red Leicester'), Cheese('Tilsit'), Cheese('Brie'), Cheese('Parmesan')]

for cheese in catalog:
  stock[cheese.kind] = cheese

sorted(stock.keys())
del catalog
sorted(stock.keys())
del cheese
sorted(stock.keys())

[]

## Limitações das referências fracas

Nem todo objeto Python pode ser o alvo - ou o referente - de uma referência fraca.
Instâncias de list e dict básicos não podem ser referentes, mas uma subclasse simples
de qualquer um deles pode resolver esse problema facilmente:

In [33]:
class MyList(list):
  """subclasse de list cujas instâncias podem ter referências fracas"""

a_list = MyList(range(10))
# a_list pode ser alvo de uma referência fraca, uma list básico não
# wref_to_a_list = weakref.ref([])
wref_to_a_list = weakref.ref(a_list)

In [35]:
# Uma tupla criada a partir de outra, na verdade, é exatamente a mesma tupla
t1 = (1, 2, 3)
t2 = tuple(t1)
t2 is t1

t3 = t1[:]
t3 is t1

True

In [41]:
# Strings literais podem criar objetos compartilhados
t1 = (1, 2, 3)
t3 = (1, 2, 3)
t3 is t1

s1 = 'ABC'
s2 = 'ABC'
s2 is s1

True